In [17]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
#OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Data Ingestion Techniques
loader - PyPDFLoader

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
reader = PyPDFLoader(file_path="../data/AI_Governance.pdf")
doc = reader.load()
doc

# Chunking or Splitting...

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splitted_doc = text_splitter.split_documents(documents=doc)
splitted_doc

# Embeddings
Vector Store - LanceDB

In [20]:
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import LanceDB
db = LanceDB.from_documents(documents=splitted_doc, embedding=OpenAIEmbeddings())

## Integrate LLM

In [21]:
from langchain_openai import OpenAI
llm = OpenAI(
    model="gpt-3.5-turbo-instruct",
    temperature=0,
    max_retries=2
)

## Design Prompt Template

In [22]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
Answer the following question based on the provided context.
Think step by step before providing a detailed answer.
<context>
{context}
</context>                                                                                                                                                                
Question: {input}
""")

## Chain
create stuff document chain

In [23]:
from langchain.chains.combine_documents.stuff import create_stuff_documents_chain
document_chain = create_stuff_documents_chain(llm=llm,prompt=prompt)

## Retrievers

In [24]:
retriever = db.as_retriever(search_type="mmr")

### Retrieval chain(Retriever + Document chain):
    Retrieval chain takes in a user query, which is then passed to the retriever to fetch relevant documents. Those documents and original inputs are then passed to an LLM to generate a response.


In [25]:
from langchain.chains import create_retrieval_chain
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [30]:
response = retrieval_chain.invoke({"input": "Give me summary of the document?"})

In [31]:
response['answer']

"\nThe document discusses the relationship between AI governance and human rights, highlighting the need for human rights principles in governing AI. It also discusses the processes and remedies for AI governance and provides recommendations. The document also includes a case study of a US school district's use of an AI system for teacher evaluations, which resulted in a violation of teachers' constitutional rights. It also mentions a decision by Finland's National Non-Discrimination and Equality Tribunal regarding a credit institution's use of AI in decision-making. The document also references the discussion of China's social credit system in a separate publication."